# Leitura da base da dados

### 1. Upside Capture

O Upside Capture Ratio é uma métrica que mede a capacidade de um fundo ou estratégia de capturar os movimentos positivos do mercado em relação ao seu benchmark. Um ratio acima de 100% indica que o fundo tende a superar o benchmark em períodos de alta, enquanto um valor abaixo sugere underperformance nesses cenários.

#### 1.1 Filtre os Períodos de Alta do Benchmark
Identifique os períodos em que o benchmark teve retornos positivos:
$$R_{bench} > 0$$

#### 1.2 Calcule o Upside Capture Ratio
A fórmula é:
    $$Upside\ Capture\ Ratio =    \frac{média\ dos\ retornos\ do\ fundo\ nos\ periodos\ de\ alta\ do\ benchmar}{média\ dos\ retornos\ de\ alta\ do\ benchmark}$$

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [32]:
dir = r'C:\Users\dieggo.araujo\Documents\Analise_FIA_Capture_Rateio\dados_raw\Retorno dos Fatores 2023 a 2024.xlsx'

In [40]:
# Função para calcular Upside e Downside Capture Ratios
def calculate_capture_ratios(fund_returns, bench_returns):
    # Upside Capture
    up_mask = bench_returns > 0
    upside_fund = fund_returns[up_mask]
    upside_bench = bench_returns[up_mask]
    upside_ratio = (np.mean(upside_fund) / np.mean(upside_bench)) * 100 if len(upside_bench) > 0 else np.nan
    
    # Downside Capture
    down_mask = bench_returns < 0
    downside_fund = fund_returns[down_mask]
    downside_bench = bench_returns[down_mask]
    downside_ratio = (np.mean(downside_fund) / np.mean(downside_bench)) * 100 if len(downside_bench) > 0 else np.nan
    
    return upside_ratio, downside_ratio

# Lista de fatores e dicionário de saída
fatores = ['Defensive', 'Valor', 'Momentum', 'Growth', 'SB']
bench = 'IBOV'
fatores_dict = {}

# Supondo que cada planilha tem colunas 'Retorno_Fundo' e 'Retorno_Benchmark'
for fator in fatores:
    df = pd.read_excel(dir, sheet_name=fator)
    
    # Calcula os ratios (ajuste os nomes das colunas conforme seu arquivo)
    upside, downside = calculate_capture_ratios(
        df[fator].values,
        df[bench].values
    )
    
    # Armazena no dicionário
    fatores_dict[fator] = {
        'dataframe': df,  # Opcional: manter o DataFrame original
        'upside_ratio': upside,
        'downside_ratio': downside,
        'capture_ratio': upside / downside if downside != 0 else np.nan,  # Índice combinado
        'spread': upside - downside
    }
    


table_data = []
for fator, metrics in fatores_dict.items():
    table_data.append([
        fator,
        f"{metrics['upside_ratio']:.2f}%",
        f"{metrics['downside_ratio']:.2f}%",
        f"{metrics['capture_ratio']:.2f}",
        f'{metrics['spread']:.2f}'
    ])

headers = ["Fator", "Upside", "Downside", "Capture Ratio", "spread"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+-----------+----------+------------+-----------------+----------+
| Fator     | Upside   | Downside   |   Capture Ratio |   spread |
+===========+==========+============+=================+==========+
| Defensive | 87.98%   | 79.84%     |            1.1  |     8.14 |
+-----------+----------+------------+-----------------+----------+
| Valor     | 101.92%  | 96.29%     |            1.06 |     5.63 |
+-----------+----------+------------+-----------------+----------+
| Momentum  | 97.03%   | 89.45%     |            1.08 |     7.58 |
+-----------+----------+------------+-----------------+----------+
| Growth    | 102.00%  | 93.66%     |            1.09 |     8.34 |
+-----------+----------+------------+-----------------+----------+
| SB        | 97.23%   | 87.53%     |            1.11 |     9.7  |
+-----------+----------+------------+-----------------+----------+
